In [1]:
import sklearn
import pandas as pd
import numpy as np
import nltk
import sklearn
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib
from sklearn.model_selection import cross_val_score
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.model_selection import train_test_split
import re
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import pickle
from wordcloud import WordCloud

In [2]:
full_labels = pd.read_csv('../input_data/full_datasets/fully_labelled_texts.csv')

In [3]:
full_labels

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,Niets gevonden,Niets gevonden
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,Niets gevonden,Niets gevonden
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,Niets gevonden,Niets gevonden
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,Niets gevonden,Niets gevonden
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,Niets gevonden,Niets gevonden
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Rookte,Ja,Nee
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,Niets gevonden,Niets gevonden
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,Niets gevonden,Niets gevonden


In [4]:
import pickle
smoking_model = pickle.load(open('../data_exploration/best_models/best_smoking_ex1.pkl', 'rb'))
drinking_model = pickle.load(open('../data_exploration/best_models/best_drinking_ex1.pkl', 'rb'))
drugs_model = pickle.load(open('../data_exploration/best_models/best_drugs_ex1.pkl', 'rb'))

In [5]:
stemmer = SnowballStemmer("dutch")
np.random.seed(500)

def create_preprocess_corpus(input_corpus, content_name, label_name, smoking=False, second_experiment=False, skip_rows=[]):
    corpus = input_corpus[['Unnamed: 0', content_name, label_name]]
    corpus = corpus.rename({content_name:'text', label_name: 'label'}, axis=1)
    corpus['processed_text'] = corpus['text'].str.replace('\t',' ')
    corpus.drop_duplicates(inplace=True)
    corpus['processed_text'] = corpus['processed_text'].astype(str)
    corpus['processed_text'] = corpus['processed_text'].str.lower()
    corpus['processed_text'] = [stemmer.stem(text) for text in corpus['processed_text']]
    if second_experiment:
        if smoking:
            replace_text = 'Rookt niet'
            corpus['label'] = corpus['label'].str.replace('Niets gevonden', 'Rookt niet')
            corpus['label'] = corpus['label'].str.replace('Rookte', 'Rookt niet')
        else:
            corpus['label'] = corpus['label'].str.replace('Niets gevonden', 'Nee')
        
    corpus = corpus.drop(corpus[corpus.label == '--'].index)
    corpus = corpus.drop(corpus[corpus.label == 'Onbekend'].index)
    corpus = corpus[~corpus.processed_text.str.contains('vertrouwelijk')]
    corpus_backup = corpus.copy()
    return corpus, corpus_backup

def add_processed_text(input_corpus, text_column):
    corpus = input_corpus.copy()
    corpus['processed_text'] = corpus[text_column].str.replace('\t',' ')
    corpus.drop_duplicates(inplace=True)
    corpus['processed_text'] = corpus['processed_text'].astype(str)
    corpus['processed_text'] = corpus['processed_text'].str.lower()
    corpus['processed_text'] = [stemmer.stem(text) for text in corpus['processed_text']]
    return corpus

In [6]:
smoking_corpus, _ = create_preprocess_corpus(full_labels, 'content', 'roken_answer_label', smoking=True)
drinking_corpus, _ = create_preprocess_corpus(full_labels, 'content', 'alcohol_answer_label')
drugs_corpus, _ = create_preprocess_corpus(full_labels, 'content', 'drugs_answer_label')

In [7]:
smoking_corpus

,Unnamed: 0,text,label,processed_text
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,"indicatie: x-bekken, x-heup axiaal klinishe ..."
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,"anamnese: geen klachten van pijn op de borst,..."
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,   [initials] [lastname] [streetname] e...
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,"geachte collega, bovengenoemde patient was op..."
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,"geachte collega, bovengenoemde patiente was o..."
...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,beloop: beloop: tc op verzoek patient beleid...
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Rookte,anamnese: verkort consult: voorgeschiedenis ...
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,beloop: beloop: zie mri lang traject 7 cm ext...
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,samenvatting: voorgeschiedenis: blanco. medic...


In [8]:



drinking_corpus

,Unnamed: 0,text,label,processed_text
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,"indicatie: x-bekken, x-heup axiaal klinishe ..."
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,"anamnese: geen klachten van pijn op de borst,..."
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,   [initials] [lastname] [streetname] e...
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,"geachte collega, bovengenoemde patient was op..."
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,"geachte collega, bovengenoemde patiente was o..."
...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,beloop: beloop: tc op verzoek patient beleid...
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Ja,anamnese: verkort consult: voorgeschiedenis ...
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,beloop: beloop: zie mri lang traject 7 cm ext...
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,samenvatting: voorgeschiedenis: blanco. medic...


In [9]:
_, smoking_x_test, _, smoking_y_test = train_test_split(smoking_corpus['processed_text'], smoking_corpus['label'], test_size=0.2, random_state=50)
_, drinking_x_test, _, drinking_y_test = train_test_split(drinking_corpus['processed_text'], drinking_corpus['label'], test_size=0.2, random_state=50)
_, drugs_x_test, _, drugs_y_test = train_test_split(drugs_corpus['processed_text'], drugs_corpus['label'], test_size=0.2, random_state=50)

# Smoking

In [10]:
predicted_nb_df = smoking_model.decision_function(smoking_x_test)
predicted_nb_labels = smoking_model.predict(smoking_x_test)
x_test_probs = pd.DataFrame(smoking_x_test, columns=['processed_text'])
x_test_probs['nothing_found_eval'] = [x[0] for x in predicted_nb_df]
x_test_probs['current_user_eval'] = [x[1] for x in predicted_nb_df]
x_test_probs['non_user_eval'] = [x[2] for x in predicted_nb_df]
x_test_probs['previous_user_eval'] = [x[3] for x in predicted_nb_df]
x_test_probs['predicted_label'] = predicted_nb_labels
x_test_probs['roken_answer_label'] = smoking_y_test
x_test_probs = x_test_probs.merge(smoking_corpus[['text', 'processed_text']], left_on='processed_text', right_on='processed_text', how='left')

In [11]:
sorted_probs_current = x_test_probs[x_test_probs.predicted_label != 'Rookt'].sort_values(by=['current_user_eval'], ascending=False)
sorted_probs_non = x_test_probs[~x_test_probs.processed_text.isin(sorted_probs_current.head(100).processed_text)][x_test_probs.predicted_label != 'Rookt niet'].sort_values(by=['non_user_eval'], ascending=False)
texts_to_label = sorted_probs_current[['text', 'processed_text', 'roken_answer_label']].head(100)
#texts_to_label = texts_to_label.append(sorted_probs_previous[~sorted_probs_previous.processed_text.isin(texts_to_label.processed_text)][['text', 'processed_text', 'roken_answer_label']].head(67))
texts_to_label = texts_to_label.append(sorted_probs_non[~sorted_probs_non.processed_text.isin(texts_to_label.processed_text)][['text', 'processed_text', 'roken_answer_label']].head(100))
texts_to_label = texts_to_label.merge(drinking_corpus.rename(columns={'label': 'alcohol_answer_label'})[['processed_text', 'alcohol_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label = texts_to_label.merge(drugs_corpus.rename(columns={'label': 'drugs_answer_label'})[['processed_text', 'drugs_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label = texts_to_label.drop_duplicates()

/tmp/ipykernel_3459292/2195468245.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sorted_probs_non = x_test_probs[~x_test_probs.processed_text.isin(sorted_probs_current.head(100).processed_text)][x_test_probs.predicted_label != 'Rookt niet'].sort_values(by=['previous_user_eval'], ascending=False)
/tmp/ipykernel_3459292/2195468245.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts_to_label = texts_to_label.append(sorted_probs_non[~sorted_probs_non.processed_text.isin(texts_to_label.processed_text)][['text', 'processed_text', 'roken_answer_label']].head(100))


In [12]:
texts_to_label

,text,processed_text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
1,Anamnese: \t Verkort consult: 28-2-2022 Rad TC...,anamnese: verkort consult: 28-2-2022 rad tc ...,Rookte,Ja,Nee
2,Anamnese: \tRvK: pijn op de borst en palpitati...,anamnese: rvk: pijn op de borst en palpitatie...,Rookt,Ja,Nee
3,"Samenvatting: \tglaucoom ods, wv gtt \t2014 p...","samenvatting: glaucoom ods, wv gtt 2014 pse...",Niets gevonden,Niets gevonden,Niets gevonden
4,Samenvatting: \t2/1/23 vitrectomie + Avastin +...,samenvatting: 2/1/23 vitrectomie + avastin + ...,Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
195,Anamnese: \t Verkort consult: div operaties (o...,anamnese: verkort consult: div operaties (or...,Rookte,Nee,Nee
196,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Ja,Nee
197,Anamnese: \tIndicatie cmc-1 resectieAlgemene ...,anamnese: indicatie cmc-1 resectiealgemene a...,Rookte,Ja,Nee
198,Anamnese: \tIndicatie tonsillectomie boven 16...,anamnese: indicatie tonsillectomie boven 16 ...,Rookte,Nee,Nee


# Drinking

In [13]:
predicted_nb_df = drinking_model.decision_function(drinking_x_test)
predicted_nb = drinking_model.predict(drinking_x_test)
x_test_probs = pd.DataFrame(drinking_x_test, columns=['processed_text'])
x_test_probs['current_user_eval'] = [x[0] for x in predicted_nb_df]
x_test_probs['non_user_eval'] = [x[1] for x in predicted_nb_df]
x_test_probs['nothing_found_eval'] = [x[2] for x in predicted_nb_df]
x_test_probs['predicted_label'] = predicted_nb
x_test_probs['alcohol_answer_label'] = drinking_y_test
x_test_probs = x_test_probs.merge(drinking_corpus[['text', 'processed_text']], left_on='processed_text', right_on='processed_text', how='left')

In [14]:
sorted_probs_current = x_test_probs[x_test_probs.predicted_label != 'Ja'].sort_values(by=['current_user_eval'], ascending=False)
sorted_probs_non = x_test_probs[~x_test_probs.processed_text.isin(sorted_probs_current.head(100).processed_text)][x_test_probs.predicted_label != 'Nee'].sort_values(by=['non_user_eval'], ascending=False)
texts_to_label_ac = sorted_probs_current[~sorted_probs_current.processed_text.isin(texts_to_label.processed_text)][['text', 'processed_text', 'alcohol_answer_label']].drop_duplicates().head(100)
texts_to_label_ac = texts_to_label_ac.append(sorted_probs_non[(~sorted_probs_non.processed_text.isin(texts_to_label_ac.processed_text)) & (~sorted_probs_non.processed_text.isin(texts_to_label.processed_text))][['text', 'processed_text', 'alcohol_answer_label']].drop_duplicates().head(100))
texts_to_label_ac = texts_to_label_ac.merge(smoking_corpus.rename(columns={'label': 'roken_answer_label'})[['processed_text', 'roken_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label_ac = texts_to_label_ac.merge(drugs_corpus.rename(columns={'label': 'drugs_answer_label'})[['processed_text', 'drugs_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label_merged = texts_to_label.append(texts_to_label_ac, ignore_index = True)
texts_to_label_merged = texts_to_label_merged.drop_duplicates()

/tmp/ipykernel_3459292/4034884161.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sorted_probs_non = x_test_probs[~x_test_probs.processed_text.isin(sorted_probs_current.head(100).processed_text)][x_test_probs.predicted_label != 'Nee'].sort_values(by=['non_user_eval'], ascending=False)
/tmp/ipykernel_3459292/4034884161.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts_to_label_ac = texts_to_label_ac.append(sorted_probs_non[(~sorted_probs_non.processed_text.isin(texts_to_label_ac.processed_text)) & (~sorted_probs_non.processed_text.isin(texts_to_label.processed_text))][['text', 'processed_text', 'alcohol_answer_label']].drop_duplicates().head(100))
/tmp/ipykernel_3459292/4034884161.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts_to_label_merged = texts_to_labe

In [15]:
texts_to_label_merged

,text,processed_text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
1,Anamnese: \t Verkort consult: 28-2-2022 Rad TC...,anamnese: verkort consult: 28-2-2022 rad tc ...,Rookte,Ja,Nee
2,Anamnese: \tRvK: pijn op de borst en palpitati...,anamnese: rvk: pijn op de borst en palpitatie...,Rookt,Ja,Nee
3,"Samenvatting: \tglaucoom ods, wv gtt \t2014 p...","samenvatting: glaucoom ods, wv gtt 2014 pse...",Niets gevonden,Niets gevonden,Niets gevonden
4,Samenvatting: \t2/1/23 vitrectomie + Avastin +...,samenvatting: 2/1/23 vitrectomie + avastin + ...,Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
455,Anamnese: \tPatiënt gesproken nee dokterassist...,anamnese: patient gesproken nee dokterassiste...,Niets gevonden,Niets gevonden,Niets gevonden
456,Anamnese: \t Verkort consult: RvK\tCentrale ma...,anamnese: verkort consult: rvk centrale mass...,Rookte,Ja,Nee
457,Anamnese: \tNIET VERSCHENEN ZONDER BERICHT\t\t...,anamnese: niet verschenen zonder bericht ik ...,Niets gevonden,Niets gevonden,Niets gevonden
458,Conclusie: \tSamenvatting ABCDE: Reden van kom...,conclusie: samenvatting abcde: reden van koms...,Niets gevonden,Niets gevonden,Niets gevonden


# Drugs

In [16]:
predicted_nb_df = drugs_model.decision_function(drugs_x_test)
predicted_nb = drugs_model.predict(drugs_x_test)
x_test_probs = pd.DataFrame(drugs_x_test, columns=['processed_text'])
x_test_probs['current_user_eval'] = [x[0] for x in predicted_nb_df]
x_test_probs['non_user_eval'] = [x[1] for x in predicted_nb_df]
x_test_probs['nothing_found_eval'] = [x[2] for x in predicted_nb_df]
x_test_probs['predicted_label'] = predicted_nb
x_test_probs['drugs_answer_label'] = drugs_y_test
x_test_probs = x_test_probs.merge(drugs_corpus[['text', 'processed_text']], left_on='processed_text', right_on='processed_text', how='left')

In [17]:
sorted_probs_current = x_test_probs[x_test_probs.predicted_label != 'Ja'].sort_values(by=['current_user_eval'], ascending=False)
sorted_probs_non = x_test_probs[~x_test_probs.processed_text.isin(sorted_probs_current.head(100).processed_text)][x_test_probs.predicted_label != 'Nee'].sort_values(by=['non_user_eval'], ascending=False)
texts_to_label_dr = sorted_probs_current[['text', 'processed_text', 'drugs_answer_label']][~sorted_probs_current.processed_text.isin(texts_to_label_merged.processed_text)].head(100)
texts_to_label_dr = texts_to_label_dr.append(sorted_probs_non[(~sorted_probs_non.processed_text.isin(texts_to_label_dr.processed_text)) & (~sorted_probs_non.processed_text.isin(texts_to_label_merged.processed_text))][['text', 'processed_text', 'drugs_answer_label']].drop_duplicates().head(100))
texts_to_label_dr = texts_to_label_dr.merge(smoking_corpus.rename(columns={'label': 'roken_answer_label'})[['processed_text', 'roken_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label_dr = texts_to_label_dr.merge(drinking_corpus.rename(columns={'label': 'alcohol_answer_label'})[['processed_text', 'alcohol_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label_merged_merged = texts_to_label_merged.append(texts_to_label_dr, ignore_index = True)
texts_to_label_merged_merged = texts_to_label_merged_merged.drop_duplicates()

/tmp/ipykernel_3459292/145590820.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sorted_probs_non = x_test_probs[~x_test_probs.processed_text.isin(sorted_probs_current.head(100).processed_text)][x_test_probs.predicted_label != 'Nee'].sort_values(by=['non_user_eval'], ascending=False)
/tmp/ipykernel_3459292/145590820.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts_to_label_dr = texts_to_label_dr.append(sorted_probs_non[(~sorted_probs_non.processed_text.isin(texts_to_label_dr.processed_text)) & (~sorted_probs_non.processed_text.isin(texts_to_label_merged.processed_text))][['text', 'processed_text', 'drugs_answer_label']].drop_duplicates().head(100))
/tmp/ipykernel_3459292/145590820.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts_to_label_merged_merged = text

In [18]:
texts_to_label_merged_merged

,text,processed_text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
1,Anamnese: \t Verkort consult: 28-2-2022 Rad TC...,anamnese: verkort consult: 28-2-2022 rad tc ...,Rookte,Ja,Nee
2,Anamnese: \tRvK: pijn op de borst en palpitati...,anamnese: rvk: pijn op de borst en palpitatie...,Rookt,Ja,Nee
3,"Samenvatting: \tglaucoom ods, wv gtt \t2014 p...","samenvatting: glaucoom ods, wv gtt 2014 pse...",Niets gevonden,Niets gevonden,Niets gevonden
4,Samenvatting: \t2/1/23 vitrectomie + Avastin +...,samenvatting: 2/1/23 vitrectomie + avastin + ...,Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
595,Anamnese: \tHuisarts vraag op opvolging\t\tIk...,anamnese: huisarts vraag op opvolging ik he...,Niets gevonden,Niets gevonden,Niets gevonden
596,Beleid: \tDr. Jansweijer- ontslag naar huis me...,beleid: dr. jansweijer- ontslag naar huis met...,Niets gevonden,Niets gevonden,Niets gevonden
597,"Geachte Mw.Dr. A.E. v/d Meulen-de Jong, MDL-ar...","geachte mw.dr. a.e. v/d meulen-de jong, mdl-ar...",Niets gevonden,Niets gevonden,Niets gevonden
598,Anamnese: \tDerbij geroepen. patient zat te ga...,anamnese: derbij geroepen. patient zat te gas...,Niets gevonden,Niets gevonden,Niets gevonden


In [19]:
texts_to_label = texts_to_label_merged_merged.drop_duplicates()

In [20]:
texts_to_label

,text,processed_text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Anamnese: \t Verkort consult: Aandachtspunten ...,anamnese: verkort consult: aandachtspunten u...,Rookte,Nee,Nee
1,Anamnese: \t Verkort consult: 28-2-2022 Rad TC...,anamnese: verkort consult: 28-2-2022 rad tc ...,Rookte,Ja,Nee
2,Anamnese: \tRvK: pijn op de borst en palpitati...,anamnese: rvk: pijn op de borst en palpitatie...,Rookt,Ja,Nee
3,"Samenvatting: \tglaucoom ods, wv gtt \t2014 p...","samenvatting: glaucoom ods, wv gtt 2014 pse...",Niets gevonden,Niets gevonden,Niets gevonden
4,Samenvatting: \t2/1/23 vitrectomie + Avastin +...,samenvatting: 2/1/23 vitrectomie + avastin + ...,Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
595,Anamnese: \tHuisarts vraag op opvolging\t\tIk...,anamnese: huisarts vraag op opvolging ik he...,Niets gevonden,Niets gevonden,Niets gevonden
596,Beleid: \tDr. Jansweijer- ontslag naar huis me...,beleid: dr. jansweijer- ontslag naar huis met...,Niets gevonden,Niets gevonden,Niets gevonden
597,"Geachte Mw.Dr. A.E. v/d Meulen-de Jong, MDL-ar...","geachte mw.dr. a.e. v/d meulen-de jong, mdl-ar...",Niets gevonden,Niets gevonden,Niets gevonden
598,Anamnese: \tDerbij geroepen. patient zat te ga...,anamnese: derbij geroepen. patient zat te gas...,Niets gevonden,Niets gevonden,Niets gevonden


In [21]:
full_labels

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,Niets gevonden,Niets gevonden
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,Niets gevonden,Niets gevonden
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,Niets gevonden,Niets gevonden
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,Niets gevonden,Niets gevonden
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,Niets gevonden,Niets gevonden
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Rookte,Ja,Nee
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,Niets gevonden,Niets gevonden
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,Niets gevonden,Niets gevonden


In [22]:
full_labels_no_dup = full_labels[~full_labels.content.isin(texts_to_label.text)]

In [23]:
full_labels_no_dup

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,Niets gevonden,Niets gevonden
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,Niets gevonden,Niets gevonden
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,Niets gevonden,Niets gevonden
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,Niets gevonden,Niets gevonden
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,Niets gevonden,Niets gevonden
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Rookte,Ja,Nee
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,Niets gevonden,Niets gevonden
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,Niets gevonden,Niets gevonden


In [24]:
random_samples = full_labels_no_dup.sample(frac=1).head(1000 - len(texts_to_label.index))

In [25]:
full_samples = add_processed_text(random_samples, 'content')

In [26]:
full_samples

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label,processed_text
56324,643693,Geachte heer Van der [LASTNAME]\t\t\tAllereers...,Niets gevonden,Niets gevonden,Niets gevonden,geachte heer van der [lastname] allereerst g...
32989,560050,"Geachte collega,\t\tBovengenoemde patiënt zage...",Niets gevonden,Niets gevonden,Niets gevonden,"geachte collega, bovengenoemde patient zagen ..."
44134,605990,Anamnese: \t Verkort consult: Aandachtspunten ...,Rookte,Ja,Nee,anamnese: verkort consult: aandachtspunten u...
19204,447355, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,Niets gevonden,Niets gevonden,   [initials] [lastname] [streetname] a...
7116,365395,"Geachte collega,\t\tBovengenoemde patiënte wer...",Niets gevonden,Niets gevonden,Niets gevonden,"geachte collega, bovengenoemde patiente werd ..."
...,...,...,...,...,...,...
34413,566308,Beloop: \tBeloop: Telefonisch contact met moed...,Niets gevonden,Niets gevonden,Niets gevonden,beloop: beloop: telefonisch contact met moede...
40140,591056,Indicatie:\tcontrole\t\tKlinishe gegevens:\tOx...,Niets gevonden,Niets gevonden,Niets gevonden,indicatie: controle klinishe gegevens: oxford...
15742,430659,Anamnese: \t Verkort consult: kan een beetje n...,Rookte,Nee,Nee,anamnese: verkort consult: kan een beetje ne...
10795,399503,Indicatie:\tZie ingescande aanvraag\t\tKlinish...,Niets gevonden,Niets gevonden,Niets gevonden,indicatie: zie ingescande aanvraag klinishe g...


In [27]:
text	processed_text	roken_answer_label	alcohol_answer_label	drugs_answer_label

SyntaxError: invalid syntax (323927493.py, line 1)

In [28]:
full_texts_to_label = texts_to_label[['text', 'roken_answer_label', 'alcohol_answer_label', 'drugs_answer_label']].append(full_samples.rename(columns={'content':'text'})[['text', 'roken_answer_label', 'alcohol_answer_label', 'drugs_answer_label']])

/tmp/ipykernel_3459292/923319018.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_texts_to_label = texts_to_label[['text', 'roken_answer_label', 'alcohol_answer_label', 'drugs_answer_label']].append(full_samples.rename(columns={'content':'text'})[['text', 'roken_answer_label', 'alcohol_answer_label', 'drugs_answer_label']])


In [29]:
full_texts_to_label

,text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Anamnese: \t Verkort consult: Aandachtspunten ...,Rookte,Nee,Nee
1,Anamnese: \t Verkort consult: 28-2-2022 Rad TC...,Rookte,Ja,Nee
2,Anamnese: \tRvK: pijn op de borst en palpitati...,Rookt,Ja,Nee
3,"Samenvatting: \tglaucoom ods, wv gtt \t2014 p...",Niets gevonden,Niets gevonden,Niets gevonden
4,Samenvatting: \t2/1/23 vitrectomie + Avastin +...,Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...
34413,Beloop: \tBeloop: Telefonisch contact met moed...,Niets gevonden,Niets gevonden,Niets gevonden
40140,Indicatie:\tcontrole\t\tKlinishe gegevens:\tOx...,Niets gevonden,Niets gevonden,Niets gevonden
15742,Anamnese: \t Verkort consult: kan een beetje n...,Rookte,Nee,Nee
10795,Indicatie:\tZie ingescande aanvraag\t\tKlinish...,Niets gevonden,Niets gevonden,Niets gevonden


In [34]:
full_texts_to_label.to_csv('full_texts_to_label_full4.csv')

In [30]:
full_labels_no_dup.sample(frac=1).head(1)

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
56849,644975,"Samenvatting: \tElectief, 3VD \t\t06-05-2022 C...",Niets gevonden,Niets gevonden,Niets gevonden


In [31]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [32]:
full_labels_no_dup.sample(frac=1).head(1)

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
47608,619040,"Samenvatting: \t06-2021 pterygium excisie OD\t\t\tAnamnese: \tgaat goed.\t\thylan zn \t\t\tLichamelijk onderzoek: \tLichamelijk onderzoekOogdruk OD: 18 mmHg, applanatie. Oogdruk OS: 18 mmHg, applanatie. Spleetlamp OD: inferior punctata, voorste oogkamer matig diepgonio tm 3-4 kw a vue. Spleetlamp OS: geen pterygium, voorste oogkamer matig diepgonio tm 3-4 kw a vue. Fundoscopie ODS: scherp begrensde papillen, niet suspect. \t\tConclusie: \tConclusie: iets sicca nog bij traanfilm insufficiëntie \t\tBeleid: \tBeleid: herstart vidisic \thylan door \t\t\tOverige acties: \tAfspraak maken: Controle afspraak over 1j. Opmerking: CPTOA Steen .",Niets gevonden,Niets gevonden,Niets gevonden
